In [8]:
import pickle
import glob
import pandas as pd
import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import log_loss
import lightgbm as lgb

# import time
# print('sleeping')
# time.sleep(7200)
# print('sleep done =======================')
for feat in sorted(glob.glob('../features/*.pkl')):
    if '3_feat' in feat:
        continue
    print("'{}',".format(feat))

'../features/fasttext_cnn2d_5_feat.pkl',
'../features/fasttext_cnn_gru_5_feat.pkl',
'../features/fasttext_cnn_v1_5_feat.pkl',
'../features/fasttext_cnn_v2_5_feat.pkl',
'../features/fasttext_cudnn_gru_5_feat.pkl',
'../features/fasttext_gru_v1_5_feat.pkl',
'../features/fasttext_lstm_v1_5_feat.pkl',
'../features/glove_cnn2d_5_feat.pkl',
'../features/glove_cnn_gru_5_feat.pkl',
'../features/glove_cnn_v1_5_feat.pkl',
'../features/glove_cnn_v2_5_feat.pkl',
'../features/glove_cudnn_gru_5_feat.pkl',
'../features/glove_gru_v1_5_feat.pkl',
'../features/glove_lstm_v1_5_feat.pkl',
'../features/lgb1_feat.pkl',
'../features/lr_feat1.pkl',
'../features/lr_feat2.pkl',
'../features/lstm_attention_fasttext_10_feat.pkl',
'../features/lstm_attention_fasttext_4_feat.pkl',
'../features/lstm_attention_fasttext_adj2_4_feat.pkl',
'../features/lstm_attention_glove_5_feat.pkl',
'../features/mnb_feat1.pkl',
'../features/mnb_feat2.pkl',
'../features/muse_cnn2d_5_feat.pkl',
'../features/muse_cnn_gru_5_feat.pkl',
'..

In [9]:
# load feats
train_x,test_x = [],[]

feats_files = [
'../features/fasttext_cnn2d_5_feat.pkl',
'../features/fasttext_cnn_gru_5_feat.pkl',
'../features/fasttext_cnn_v1_5_feat.pkl',
'../features/fasttext_cnn_v2_5_feat.pkl',
'../features/fasttext_cudnn_gru_5_feat.pkl',
#'../features/fasttext_gru_v1_5_feat.pkl',
'../features/fasttext_lstm_v1_5_feat.pkl',
'../features/glove_cnn2d_5_feat.pkl',
'../features/glove_cnn_gru_5_feat.pkl',
'../features/glove_cnn_v1_5_feat.pkl',
'../features/glove_cnn_v2_5_feat.pkl',
'../features/glove_cudnn_gru_5_feat.pkl',
#'../features/glove_gru_v1_5_feat.pkl',
#'../features/glove_lstm_v1_5_feat.pkl',
'../features/lgb1_feat.pkl',
'../features/lr_feat1.pkl',
'../features/lr_feat2.pkl',
# '../features/lstm_attention_fasttext_10_feat.pkl',
# '../features/lstm_attention_fasttext_4_feat.pkl',
'../features/lstm_attention_fasttext_adj2_4_feat.pkl',
'../features/lstm_attention_glove_5_feat.pkl',
'../features/mnb_feat1.pkl',
'../features/mnb_feat2.pkl',
'../features/muse_cnn2d_5_feat.pkl',
'../features/muse_cnn_gru_5_feat.pkl',
#'../features/muse_cnn_v1_5_feat.pkl',
'../features/muse_cnn_v2_5_feat.pkl',
'../features/muse_cudnn_gru_5_feat.pkl',
#'../features/muse_gru_v1_5_feat.pkl',
#'../features/muse_lstm_v1_5_feat.pkl',
'../features/no_pretrained_cnn2d_5_feat.pkl',
'../features/no_pretrained_cnn_gru_5_feat.pkl',
'../features/no_pretrained_cnn_v1_5_feat.pkl',
'../features/no_pretrained_cnn_v2_5_feat.pkl',
'../features/no_pretrained_cudnn_gru_5_feat.pkl',
'../features/no_pretrained_gru_v1_5_feat.pkl',
'../features/no_pretrained_lstm_v1_5_feat.pkl',
'../features/other_feat.pkl',
#'../features/pool_gru_fasttext_5_feat.pkl',
'../features/pool_gru_fasttext_adj1_10_feat.pkl',
'../features/pool_gru_fasttext_adj2_10_feat.pkl',
#'../features/pool_gru_fasttext_adj2_5_feat.pkl',
'../features/pool_gru_glove_5_feat.pkl',
'../features/ridge_feat1.pkl',
'../features/ridge_feat2.pkl',
# '../features/tfidf_feat1.pkl',
# '../features/tfidf_feat2.pkl',
'../features/tilli_lr_feat.pkl',
'../features/wordbatch_feat.pkl',
]

for feat in feats_files:
    print('file path',feat)
    a,b = pickle.load(open(feat,'rb'))
    print(a.shape,b.shape)
    train_x.append(a)
    test_x.append(b)
train_x = np.nan_to_num(np.hstack(train_x))
test_x = np.nan_to_num(np.hstack(test_x))
print(train_x.shape)
    
# load y
train = pd.read_csv("../input/train.csv")
list_classes = ["toxic", "severe_toxic", "obscene", "threat", "insult", "identity_hate"]
train_y = train[list_classes].values.astype('int')
print(train_x.shape)
print(train_x[0])

file path ../features/fasttext_cnn2d_5_feat.pkl
(159571, 6) (153164, 6)
file path ../features/fasttext_cnn_gru_5_feat.pkl
(159571, 6) (153164, 6)
file path ../features/fasttext_cnn_v1_5_feat.pkl
(159571, 6) (153164, 6)
file path ../features/fasttext_cnn_v2_5_feat.pkl
(159571, 6) (153164, 6)
file path ../features/fasttext_cudnn_gru_5_feat.pkl
(159571, 6) (153164, 6)
file path ../features/fasttext_lstm_v1_5_feat.pkl
(159571, 6) (153164, 6)
file path ../features/glove_cnn2d_5_feat.pkl
(159571, 6) (153164, 6)
file path ../features/glove_cnn_gru_5_feat.pkl
(159571, 6) (153164, 6)
file path ../features/glove_cnn_v1_5_feat.pkl
(159571, 6) (153164, 6)
file path ../features/glove_cnn_v2_5_feat.pkl
(159571, 6) (153164, 6)
file path ../features/glove_cudnn_gru_5_feat.pkl
(159571, 6) (153164, 6)
file path ../features/lgb1_feat.pkl
(159571, 6) (153164, 6)
file path ../features/lr_feat1.pkl
(159571, 6) (153164, 6)
file path ../features/lr_feat2.pkl
(159571, 6) (153164, 6)
file path ../features/lstm_

In [10]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
train_x = scaler.fit_transform(train_x)
test_x = scaler.transform(test_x)
print(train_x[0])

[3.20521454e-05 4.89555083e-12 1.90365273e-07 4.64348831e-15
 3.15957909e-07 2.59789556e-10 9.16498370e-05 1.13355713e-06
 7.24869416e-05 5.81399818e-07 2.43423178e-05 4.81439416e-06
 1.07179477e-03 4.88805527e-13 4.95880057e-06 3.81615681e-09
 2.18490891e-06 4.19219603e-08 2.31894219e-03 1.29394237e-08
 7.93581931e-05 3.53740753e-08 2.54700022e-05 7.03738971e-07
 3.88531271e-04 9.67030148e-10 7.92067129e-06 8.69702087e-09
 3.12861708e-06 6.02284241e-08 2.64098439e-04 6.85468495e-07
 2.83630892e-04 2.00738290e-07 3.37293069e-05 5.24582129e-06
 7.83401448e-03 3.65120710e-10 1.42477453e-04 2.12124282e-08
 3.13741853e-05 5.19897572e-07 3.44083674e-03 1.21799667e-05
 2.48687941e-03 1.46833189e-06 4.65022698e-04 7.12203760e-05
 1.04952266e-03 3.07052981e-10 5.30055256e-04 1.19123942e-09
 5.69250008e-06 7.48166902e-08 1.29255704e-02 5.85622927e-10
 1.30088318e-04 4.54351489e-08 9.31599431e-04 1.13912413e-06
 4.09279057e-04 1.84011858e-08 1.15490292e-05 6.51481030e-07
 7.78886272e-06 3.721687

In [11]:
from sklearn.model_selection import KFold, StratifiedKFold
from sklearn.metrics import roc_auc_score
def simple_ens(model_name,k=3,rnd=233,lr=0.05,feature_fraction=0.9,bagging_fraction=0.9,
               bag_frec=3,met='binary_logloss',max_d=3):
    kf = StratifiedKFold(n_splits=k, shuffle=True, random_state=rnd)
    test_pred = np.zeros((153164,6))
    cls_auc_res = [0,0,0,0,0,0]
    all_train_loss_l,all_val_loss_l = 0,0
    all_train_auc_l,all_val_auc_l = 0,0
    
    for i in range(6):
        val_loss_l,train_loss_l = 0,0
        val_auc_l,train_auc_l = 0,0
        fold_cnt = 0
        for train_index, test_index in kf.split(train_x,train_y[:,i]):
            # x,y
            curr_x,curr_y = train_x[train_index],train_y[train_index]
            hold_out_x,hold_out_y = train_x[test_index],train_y[test_index]
            d_test = test_x

            # share params
            params = {
                    'application': 'binary',
                    #'num_leaves': 8,
                    #'lambda_l1': 1,
                    'lambda_l2': 1.0,
                    'max_depth': max_d,
                    'metric': met, # or auc
                    'data_random_seed': 2,
                    'learning_rate':lr,
                    # 'bagging_fraction': bagging_fraction,
                    # 'bagging_freq':bag_frec,
                    'feature_fraction': feature_fraction,

                    }
            if met == 'auc':
                s_round = 60
            else:
                s_round = 30
            # train for each class
            d_train = lgb.Dataset(curr_x, curr_y[:,i])
            d_valid = lgb.Dataset(hold_out_x, hold_out_y[:,i])
            watchlist = [d_train, d_valid]
            model = lgb.train(params,
                      train_set=d_train,
                      num_boost_round=2000,
                      valid_sets=watchlist,
                      early_stopping_rounds=s_round,
                      verbose_eval=None)
            print(fold_cnt,'fold: ',end='')
            fold_cnt += 1
            try:
                train_pred = model.predict(curr_x)
                tmp_test_pred = model.predict(hold_out_x)
                
                curr_train_loss = log_loss(curr_y[:,i],train_pred)
                curr_val_loss = log_loss(hold_out_y[:,i],tmp_test_pred)
                
                curr_train_auc = roc_auc_score(curr_y[:,i],train_pred)
                curr_val_auc = roc_auc_score(hold_out_y[:,i],tmp_test_pred)
                
                print('ls',curr_train_loss,curr_val_loss,'auc',curr_train_auc,curr_val_auc)
                val_loss_l += curr_val_loss
                train_loss_l += curr_train_loss
                val_auc_l += curr_val_auc
                train_auc_l += curr_train_auc
            except:
                pass
            curr_test_pred = model.predict(d_test)
            test_pred[:,i] += curr_test_pred
            
        # avg k fold
        train_loss_l = train_loss_l/k
        val_loss_l = val_loss_l/k
        train_auc_l = train_auc_l/k
        val_auc_l = val_auc_l/k
        print(list_classes[i], lr, feature_fraction, max_d)
        print('this class avg train',train_loss_l,'avg val',val_loss_l)
        print('this class auc train',train_auc_l,'auc val',val_auc_l)
        cls_auc_res[i] = val_auc_l
        
        
        # avg 6 class
        all_train_loss_l += train_loss_l/6
        all_val_loss_l += val_loss_l/6
        all_train_auc_l += train_auc_l/6
        all_val_auc_l += val_auc_l/6
        print('========================')
    test_pred = test_pred/k
    print('all loss avg',all_train_loss_l,all_val_loss_l)
    print('all auc avg',all_train_auc_l,all_val_auc_l)
    print('=======================================================')
    return test_pred, cls_auc_res

print('done')

done


In [12]:
%%time
lgb_res,_ = simple_ens('lgb',10,666,0.05,0.6)
sample_submission = pd.read_csv("../input/sample_submission.csv")
sample_submission[list_classes] = lgb_res
sample_submission.to_csv("../results/lgb_some_fold10.gz", index=False, compression='gzip')
print(sample_submission.head())
print('save done')

# select feats
# all loss avg 0.0323893274343 0.0361490549659
# all auc avg 0.994298774643 0.991023792329

# 1 fold
# rf tfidf select feats, feat_fraction 0.6 this fold avg train 0.0324570163191 avg val 0.0365732819959
# rf tfidf select feats, feat_fraction 0.75 this fold avg train 0.0325535548896 avg val 0.0366403197325
# rf tfidf select feats, feat_fraction 0.5 this fold avg train 0.0328270646453 avg val 0.0366173320613

# rm muse, rm ft, glove gru lstm , feat frac 0.6, min max scalar
# all loss avg 0.03185315549604115 0.03507562597568142 all auc avg 0.9945693363386173 0.9922054290233556

# select rm based on analyse_corr, feat frac 0.6, min max scalar
# all loss avg 0.031793628328771764 0.03509315947756795 all auc avg 0.9946176237713884 0.9921554075579672

0 fold: ls 0.07006243401594699 0.07815096070482307 auc 0.9905916359258318 0.9871330890733523
1 fold: ls 0.06927252834677794 0.07087472482786562 auc 0.9908541818460636 0.9899988856091371
2 fold: ls 0.07041427383633003 0.07557685660013301 auc 0.9904400414270562 0.9884979460779784
3 fold: ls 0.06617862164930581 0.07324331272570839 auc 0.9918566864676601 0.9890381991443653
4 fold: ls 0.07016092121569122 0.07747709516959411 auc 0.9905020567115067 0.9879844039177511
5 fold: ls 0.06744077438596482 0.07512222473078832 auc 0.9914720671511664 0.988201353628391
6 fold: ls 0.06977119458540804 0.07744602455120202 auc 0.9906751052370067 0.9879179953665417
7 fold: ls 0.06778442608177622 0.06974776246043846 auc 0.9913472906862133 0.9905525134704237
8 fold: ls 0.06637329323330114 0.07370617080865154 auc 0.9917788730773065 0.9895266229028914
9 fold: ls 0.06846744787032376 0.07655402601863402 auc 0.9910880600586808 0.988065986840766
toxic 0.05 0.6 3
this class avg train 0.06859259152208261 avg val 0.0747